In [11]:
import pandas as pd
import plotly.express as px
import os
from config import *
import math
import textwrap
from config import AllPath
import plotly.graph_objects as go

all_paths = AllPath()



In [12]:
local_project_path = all_paths.save_data_path

cities_path = local_project_path+'cities_weather_datas.csv'

target_cities_datas = pd.read_csv(cities_path)
# target_cities_datas

In [13]:
hotel_path = all_paths.save_data_path + 'hotel.json'
df = pd.read_json(hotel_path)
display(df.head(3))


,city,lat,lon,name,review_score,url,description
0,Amiens,49.891833,2.305994,Hotel Central Anzac,7.4,https://www.booking.com/hotel/fr/central-anzac...,"Centrally located, Hotel Central Anzac is a 2-..."
1,Amiens,49.891999,2.291970,B&B HOTEL Amiens Centre Cathédrale,7.7,https://www.booking.com/hotel/fr/ibis-amiens-c...,"Offering a restaurant and free Wi-Fi, B&B HOTE..."
2,Amiens,49.891676,2.307754,"Holiday Inn Express Amiens, an IHG Hotel",7.9,https://www.booking.com/hotel/fr/express-by-ho...,Holiday Inn Express offers comfortable guest r...


In [14]:
# df_merge = pd.merge(df, target_cities_datas, left_on='city', right_on='city_target', how='inner').drop(columns='city_target')
df = pd.merge(df, target_cities_datas, left_on='city', right_on='city_target', how='inner')

In [15]:
fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        color='review_score',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        color_continuous_scale=['white', 'yellow', 'red'],
                        mapbox_style='open-street-map',
                        hover_name='name',
                        hover_data=['city'],
                        # hover_data=['city', 'description'],
                        height=600)

fig.show()

In [16]:
custom_colorscale = [
    [0, 'yellow'],
    [0.33, 'orange'],
    [1, 'red']
]

trace = go.Scattermapbox(
    lat=df['lat'],
    lon=df['lon'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['review_score'],
        colorscale=custom_colorscale,
        cmin=df['review_score'].min(),
        cmax=df['review_score'].max(),
        colorbar=dict(title='Review Score')
    ),
    hoverinfo='text',
    text=df['name'] + '<br>' + df['city']+ df['description'],
)

layout = go.Layout(
    title='Map',
    mapbox=dict(
        style='carto-positron',
        center=dict(lat=46.2, lon=2.2),
        zoom=4
    ),
    height=600
)

fig = go.Figure(data=[trace], layout=layout)

fig.show()

In [17]:
def haversine(lat1, lon1, lat2, lon2):
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    R = 6371.0
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    
    return round(distance,2)

In [18]:
df['distance'] = df['distance'] = df.apply(lambda row: haversine(row['lat'], row['lon'], row['lat_target'], row['lon_target']), axis=1)

In [19]:
df.sort_values('distance')
df.sort_values('review_score').head()

,city,lat,lon,name,review_score,url,description,city_target,city_target_precipitation,date_time,city_target_temp_feels_like,city_target_wind_speed,lat_target,lon_target,city_target_weather_score,distance
219,Le-Havre,49.492303,0.122933,Hôtel Le Green,2.8,https://www.booking.com/hotel/fr/le-green.en-g...,"Located in the centre of Le Havre, this hotel ...",Le-Havre,0.2508,2024-04-06 16:48:00,12.3672,6.4784,49.4939,0.1080,0.000000,1.09
816,Bayonne,43.489623,-1.451075,hotelF1 Bayonne,5.4,https://www.booking.com/hotel/fr/hotelf1-bayon...,HotelF1 Bayonne offers accommodation a 7-minut...,Bayonne,0.0460,2024-04-06 16:48:00,17.2660,3.3132,43.4945,-1.4737,6.541405,1.90
707,Montauban,44.007347,1.385129,hotelF1 Montauban,5.4,https://www.booking.com/hotel/fr/hotelf1-monta...,"hotelF1 Montauban is situated in Montauban, 45...",Montauban,0.0400,2024-04-06 16:48:00,17.4028,4.9080,44.0176,1.3550,6.337710,2.67
197,Strasbourg,48.572891,7.792947,hotelF1 Strasbourg Pont de l'Europe,5.6,https://www.booking.com/hotel/fr/hotelf1-stras...,"Offering free WiFi, hotelF1 Strasbourg Pont de...",Strasbourg,0.1740,2024-04-06 16:48:00,17.2548,3.0220,48.5846,7.7507,6.477825,3.37
807,Bayonne,43.490490,-1.452035,Premiere Classe Bayonne,5.6,https://www.booking.com/hotel/fr/premiere-clas...,"Premiere Classe Bayonne, easily accessible fro...",Bayonne,0.0460,2024-04-06 16:48:00,17.2660,3.3132,43.4945,-1.4737,6.541405,1.80


In [20]:
df.columns

Index(['city', 'lat', 'lon', 'name', 'review_score', 'url', 'description',
       'city_target', 'city_target_precipitation', 'date_time',
       'city_target_temp_feels_like', 'city_target_wind_speed', 'lat_target',
       'lon_target', 'city_target_weather_score', 'distance'],
      dtype='object')

In [21]:

df_unique =df[['city_target','lon_target','lat_target','city_target_weather_score']].drop_duplicates()

city_df = pd.DataFrame({
    'City': df_unique['city_target'],
    'Latitude': df_unique['lat_target'],
    'Longitude': df_unique['lon_target'],
    'Rate': df_unique['city_target_weather_score']
})

hotel_df = pd.DataFrame({
    'Hotel': df['name'],
    'City': df['city'],
    'Latitude': df['lat'],
    'Longitude': df['lon'],
    'Rating' : df['review_score'],
    'Desc' : df['description'],
    'Distance' : df['distance']

})

hotel_df['Wrapped_Desc'] = ['<br>'.join(textwrap.wrap(x, width=80)) for x in hotel_df['Desc']]


In [22]:
top5_cities = df[['city','city_target_weather_score']].drop_duplicates().sort_values('city_target_weather_score',ascending=False).head()['city'].tolist()
top5_cities

['Ariege', 'Grenoble', 'Aix-en-Provence', 'Colmar', 'Avignon']

In [23]:
color_scale = [
    (0, 5, 'rgb(169,169,169)'),  
    (5, 7, 'rgb(255,255,0)'),    
    (7, 8, 'rgb(255,165,0)'),    
    (8, 9, 'rgb(255,0,0)'),      
    (9, 10, 'rgb(128,0,128)')    
]

fig = go.Figure()

for _, row in hotel_df.iterrows():
    rating = row['Rating']
    color = None
    for lower, upper, col in color_scale:
        if lower <= rating <= upper:
            color = col
            break
    hover_text = f"Hotel: {row['Hotel']}<br>Rating: {rating}<br>Description:<br><br>{row['Wrapped_Desc']}"
    # hover_text = f"Hotel: {row['Hotel']}<br>Rating: {rating}<br>Distance from (city):{row['Distance']}"

    fig.add_trace(go.Scattergeo(
        lon=[row['Longitude']],
        lat=[row['Latitude']],
        mode='markers',
        marker=dict(size=17, symbol='circle', color=color, line=dict(color='black', width=1)),
        # name=row['Hotel'],
        name='',
        visible=False,
        # hoverinfo='text',
        # text=hover_text,
        hovertemplate=hover_text,
        # hovertemplate=hover_text
        hoverlabel=dict(
            # bgcolor="skyblue",  
            bgcolor= color,  
            bordercolor="black",
            font=dict(size=12), 
            align='left',  
    )
    ))

for _, row in city_df.iterrows():
    city_name = row['City']
    if city_name in top5_cities:
        color = 'gold'
    else:
        color = 'blue'


    fig.add_trace(go.Scattergeo(
        lon=[row['Longitude']],
        lat=[row['Latitude']],
        mode='markers',
        marker=dict(size=17, symbol='circle', color=color, line=dict(color='black', width=2)),
        name='',
        visible=True,
        hovertemplate=city_name
    ))

legend_annotations = [
    dict(
        x=0,
        y=0.8,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle',
        width=90,
        height=50,
        text='Top 5',
        bgcolor='gold',
        font=dict(color='white'),
        showarrow=False
    ),
    dict(
        x=0,
        y=0.8 - 0.14,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle',
        width=90,
        height=50,
        text='Other',
        bgcolor='blue',
        font=dict(color='white'),
        showarrow=False
    )
]

legend_annotations += [
    dict(
        x=0.9,
        y=0.8 - i * 0.14,
        xref='paper',
        yref='paper',
        xanchor='left',
        yanchor='middle',
        width=90,
        height=50,
        text=f'{lower}-{upper}',
        bgcolor=color,
        font=dict(color='white'),
        showarrow=False
    ) for i, (lower, upper, color) in enumerate(color_scale)
]

# Update layout with legend annotations
fig.update_layout(
    title='Hotels in Cities',
    showlegend=False,
    geo=dict(
        scope='europe',
        center=dict(lat=46, lon=2),  
        lonaxis_range=[-5, 10], 
        lataxis_range=[42, 52]
    ),
    height=600,
    width=800,
    title_x=0.5,
    annotations=legend_annotations
)

# Add buttons for toggling between hotels and cities
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Cities', method='update',
                     args=[{'visible': [False] * len(hotel_df) + [True] * len(city_df)}])
            ],
            x=0.1,  
            y=1,  
        ),
        dict(
            type='buttons',
            buttons=[
                dict(label='Hotels', method='update',
                     args=[{'visible': [True] * len(hotel_df) + [False] * len(city_df)}])
                    #  args=[{'visible': [True] * len(hotel_df) + [True] * len(city_df)}])
            ],
            x=1, 
            y=1, 
        )
    ]
)

fig.show()

In [24]:
df.to_csv(all_paths.save_data_path+'Data_Enriched_Weather_Hotel_French_Cities.csv',index=False)

In [25]:
all_paths.save_data_path+'Data_Enriched_Weather_Hotel_French_Cities.csv'

'C:\\Users\\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak\\datas\\Data_Enriched_Weather_Hotel_French_Cities.csv'

Idea: make a weather temp background map

Idea: make a wind map direction